<a href="https://colab.research.google.com/github/FranMoritz/Projeto-Loja-Stilo-/blob/main/C%C3%B3pia_de_DadosF1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyspark

In [ ]:
#Mapear nosso Google Drive

from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Criando Sessão com o SparkSession
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("ETL Dados F1") \
    .config('spark.ui.port', '4050') \
    .config("spark.sql.csv.encoding", "UTF-8")\ #aqui define o encoding para leitura do arquivo corretamente.
    .getOrCreate()

In [ ]:
#lendo o arquivo salvo no drive
df_F1 = spark.read.csv("/content/gdrive/My Drive/projetof1/F1.csv", header=True, sep=",")

In [ ]:
df_F1.show(truncate=False)

+----------+-------------+-------------+----------------------------+---------------------+------------------------+--------+-----+----+
|date      |continent    |grand_prix   |circuit                     |winner_name          |team                    |time    |laps |year|
+----------+-------------+-------------+----------------------------+---------------------+------------------------+--------+-----+----+
|1950-05-13|Europe       |Great Britain|Silverstone Circuit         |Nino Farina          |Alfa Romeo              |02:13:23|70.0 |1950|
|1950-05-21|Europe       |Monaco       |Circuit de Monaco           |Juan Manuel Fangio   |Alfa Romeo              |03:13:18|100.0|1950|
|1950-05-30|North America|United States|Indianapolis Motor Speedway |Johnnie Parsons      |Kurtis Kraft Offenhauser|02:46:55|138.0|1950|
|1950-06-04|Europe       |Switzerland  |Circuit Bremgarten          |Nino Farina          |Alfa Romeo              |02:02:53|42.0 |1950|
|1950-06-18|Europe       |Belgium      |C

# **Aplicando Data Quality**

In [ ]:
#quantidade de linhas da tabela
total_linhas = df_F1.count()
print(f"Total de linhas: {total_linhas}")


Total de linhas: 1142


In [ ]:
#Verificando se tem nulos em todas as colunas
from pyspark.sql.functions import count, when, col, isnan

df_F1.select([
    count(when(col(c).isNull() | (col(c)=="") | isnan(c), c)).alias("nulos")
    for c in df_F1.columns
]).show()

+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|nulos|nulos|nulos|nulos|nulos|nulos|nulos|nulos|nulos|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+
|    0|    0|    0|    0|    0|    0|    0|    0|    0|
+-----+-----+-----+-----+-----+-----+-----+-----+-----+



In [ ]:
#Verificando deduplicidade
duplicados = df_F1.count() - df_F1.dropDuplicates().count()
print(f"Linhas duplicadas: {duplicados}")

Linhas duplicadas: 0


In [ ]:
#Verificando valores distintos por coluna
from pyspark.sql.functions import countDistinct, col

for c in df_F1.columns:
  print(f"Coluna {c}: {df_F1.select(countDistinct(col(c))).collect()[0][0]} valores distintos")

Coluna date: 1139 valores distintos
Coluna continent: 6 valores distintos
Coluna grand_prix: 36 valores distintos
Coluna circuit: 77 valores distintos
Coluna winner_name: 115 valores distintos
Coluna team: 65 valores distintos
Coluna time: 964 valores distintos
Coluna laps: 73 valores distintos
Coluna year: 76 valores distintos


In [ ]:
#Estatísticas descritivas básicas
df_F1.describe().show()

+-------+----------+-------------+-------------+----------------+------------------+----------+--------+------------------+------------------+
|summary|      date|    continent|   grand_prix|         circuit|       winner_name|      team|    time|              laps|              year|
+-------+----------+-------------+-------------+----------------+------------------+----------+--------+------------------+------------------+
|  count|      1142|         1142|         1142|            1142|              1142|      1142|    1142|              1142|              1142|
|   mean|      NULL|         NULL|         NULL|            NULL|              NULL|      NULL|    NULL| 64.61733800350262|1992.9991243432573|
| stddev|      NULL|         NULL|         NULL|            NULL|              NULL|      NULL|    NULL|20.065138811800857| 20.85009723923465|
|    min|1950-05-13|       Africa|    Abu Dhabi|            AVUS|       Alain Prost|Alfa Romeo|01:01:07|               1.0|              1950|

In [ ]:
#lendo o schema dos dados
df_F1.printSchema

root
 |-- date: string (nullable = true)
 |-- continent: string (nullable = true)
 |-- grand_prix: string (nullable = true)
 |-- circuit: string (nullable = true)
 |-- winner_name: string (nullable = true)
 |-- team: string (nullable = true)
 |-- time: string (nullable = true)
 |-- laps: string (nullable = true)
 |-- year: string (nullable = true)



In [ ]:
#Alterando os tipos das colunas
df_F1 = df_F1.withColumn("date" , col("date").cast("date"))
df_F1 = df_F1.withColumn("laps" , col("laps").cast("int"))
df_F1 = df_F1.withColumn("year" , col("year").cast("int"))

In [ ]:
df_F1.printSchema()

root
 |-- date: date (nullable = true)
 |-- continent: string (nullable = true)
 |-- grand_prix: string (nullable = true)
 |-- circuit: string (nullable = true)
 |-- winner_name: string (nullable = true)
 |-- team: string (nullable = true)
 |-- time: string (nullable = true)
 |-- laps: integer (nullable = true)
 |-- year: integer (nullable = true)



# **Agregações para Dashboards**

In [ ]:
#Pilotos x Qtde Vitórias
vitoria_piloto = df_F1.groupBy(col("winner_name").alias("piloto"))\
.agg(count("*").alias("total_vitorias"))\
.orderBy(col("total_vitorias").desc())

In [ ]:
vitoria_piloto.show()

+------------------+--------------+
|            piloto|total_vitorias|
+------------------+--------------+
|    Lewis Hamilton|           105|
|Michael Schumacher|            91|
|    Max Verstappen|            65|
|  Sebastian Vettel|            53|
|       Alain Prost|            51|
|      Ayrton Senna|            41|
|   Fernando Alonso|            32|
|     Nigel Mansell|            31|
|    Jackie Stewart|            27|
|         Jim Clark|            25|
|        Niki Lauda|            25|
|Juan Manuel Fangio|            24|
|      Nico Rosberg|            23|
|     Nelson Piquet|            23|
|        Damon Hill|            22|
|    Kimi Raikkonen|            21|
|     Mika Hakkinen|            20|
|     Stirling Moss|            16|
|     Jenson Button|            15|
|Emerson Fittipaldi|            14|
+------------------+--------------+
only showing top 20 rows



In [ ]:
#salvando tabala no drive
vitoria_piloto.write.csv("/content/gdrive/My Drive/projetof1/vitoria_piloto")

In [ ]:
# Vitória por pilto em cada ano
vitorias_por_ano = df_F1.groupBy("year", "Winner_name")\
.agg(count("*").alias("Vitoria_ano"))\
.orderBy("Vitoria_ano", "year", ascending=False)

In [ ]:
vitorias_por_ano.show()

+----+------------------+-----------+
|year|       Winner_name|Vitoria_ano|
+----+------------------+-----------+
|2023|    Max Verstappen|         19|
|2022|    Max Verstappen|         15|
|2013|  Sebastian Vettel|         13|
|2004|Michael Schumacher|         13|
|2020|    Lewis Hamilton|         11|
|2019|    Lewis Hamilton|         11|
|2018|    Lewis Hamilton|         11|
|2014|    Lewis Hamilton|         11|
|2011|  Sebastian Vettel|         11|
|2002|Michael Schumacher|         11|
|2021|    Max Verstappen|         10|
|2016|    Lewis Hamilton|         10|
|2015|    Lewis Hamilton|         10|
|2024|    Max Verstappen|          9|
|2017|    Lewis Hamilton|          9|
|2016|      Nico Rosberg|          9|
|2001|Michael Schumacher|          9|
|2000|Michael Schumacher|          9|
|1995|Michael Schumacher|          9|
|1992|     Nigel Mansell|          9|
+----+------------------+-----------+
only showing top 20 rows



In [ ]:
vitorias_por_ano.write.csv("/content/gdrive/My Drive/projetof1/vitorias_por_ano")

In [ ]:
#Vitorias por equipe
vitoria_equipe = df_F1.groupBy(col("team").alias("equipe"))\
.agg(count("*").alias("total_vitorias"))\
.orderBy(col("total_vitorias").desc())


In [ ]:
vitoria_equipe.show()

+--------------------+--------------+
|              equipe|total_vitorias|
+--------------------+--------------+
|             Ferrari|           249|
|            Mercedes|           121|
|    McLaren Mercedes|            85|
|    Williams Renault|            64|
|          Lotus Ford|            47|
|       McLaren Honda|            44|
|        McLaren Ford|            35|
|             Renault|            35|
|Red Bull Racing R...|            35|
|Red Bull Racing H...|            30|
|         McLaren TAG|            25|
|        Lotus Climax|            24|
|      Williams Honda|            23|
|        Tyrrell Ford|            23|
|       Williams Ford|            17|
|                 BRM|            17|
|Red Bull Racing RBPT|            17|
|Red Bull Racing H...|            16|
|        Brabham Ford|            15|
|         RBR Renault|            15|
+--------------------+--------------+
only showing top 20 rows



In [ ]:
vitoria_equipe.write.csv("/content/gdrive/My Drive/projetof1/vitoria_equipe")

In [ ]:
#Vitorias por equipe ano
vitorias_por_ano_equipe = df_F1.groupBy("year", "team")\
.agg(count("*").alias("Vitoria_ano"))\
.orderBy("Vitoria_ano", "team", ascending=False)

In [ ]:
vitorias_por_ano_equipe.show()

+----+--------------------+-----------+
|year|                team|Vitoria_ano|
+----+--------------------+-----------+
|2023|Red Bull Racing H...|         21|
|2016|            Mercedes|         19|
|2022|Red Bull Racing RBPT|         17|
|2014|            Mercedes|         16|
|2015|            Mercedes|         16|
|2019|            Mercedes|         15|
|1988|       McLaren Honda|         15|
|2004|             Ferrari|         15|
|2002|             Ferrari|         15|
|2013|Red Bull Racing R...|         13|
|2020|            Mercedes|         13|
|1996|    Williams Renault|         12|
|2011|Red Bull Racing R...|         12|
|2017|            Mercedes|         12|
|1984|         McLaren TAG|         12|
|2021|Red Bull Racing H...|         11|
|2018|            Mercedes|         11|
|2025|             McLaren|         11|
|1995|    Benetton Renault|         11|
|1992|    Williams Renault|         10|
+----+--------------------+-----------+
only showing top 20 rows



In [ ]:
vitorias_por_ano_equipe.write.csv("/content/gdrive/My Drive/projetof1/vitorias_por_ano_equipe")

In [ ]:
#Pilotos com maior vitória por circuito
vitorias_circuito = df_F1.groupBy("circuit", "Winner_name","team")\
.agg(count("*").alias("Vitoria_no_circuito"))\
.orderBy("Vitoria_no_circuito", ascending=False)

In [ ]:
vitorias_circuito.show(truncate=False)

+----------------------------------+------------------+-------------+-------------------+
|circuit                           |Winner_name       |team         |Vitoria_no_circuito|
+----------------------------------+------------------+-------------+-------------------+
|Silverstone Circuit               |Lewis Hamilton    |Mercedes     |8                  |
|Circuit de Nevers Magny Cours     |Michael Schumacher|Ferrari      |6                  |
|Imola                             |Michael Schumacher|Ferrari      |6                  |
|Circuit Gilles Villeneuve         |Michael Schumacher|Ferrari      |6                  |
|Circuit de Barcelona Catalunya    |Lewis Hamilton    |Mercedes     |6                  |
|Indianapolis Motor Speedway       |Michael Schumacher|Ferrari      |5                  |
|Sochi Autodrom                    |Lewis Hamilton    |Mercedes     |5                  |
|Autodromo Nazionale di Monza      |Michael Schumacher|Ferrari      |5                  |
|Suzuka In

In [ ]:
vitorias_circuito.write.csv("/content/gdrive/My Drive/projetof1/vitorias_circuito")

# **Book de Variáveis**

In [ ]:
df_F1.orderBy(col ("year")).show()

+----------+-------------+-------------+--------------------+--------------------+--------------------+--------+----+----+
|      date|    continent|   grand_prix|             circuit|         winner_name|                team|    time|laps|year|
+----------+-------------+-------------+--------------------+--------------------+--------------------+--------+----+----+
|1950-05-13|       Europe|Great Britain| Silverstone Circuit|         Nino Farina|          Alfa Romeo|02:13:23|  70|1950|
|1950-05-21|       Europe|       Monaco|   Circuit de Monaco|  Juan Manuel Fangio|          Alfa Romeo|03:13:18| 100|1950|
|1950-05-30|North America|United States|Indianapolis Moto...|     Johnnie Parsons|Kurtis Kraft Offe...|02:46:55| 138|1950|
|1950-06-04|       Europe|  Switzerland|  Circuit Bremgarten|         Nino Farina|          Alfa Romeo|02:02:53|  42|1950|
|1950-06-18|       Europe|      Belgium|Circuit de Spa Fr...|  Juan Manuel Fangio|          Alfa Romeo|02:47:26|  35|1950|
|1950-07-02|    

In [ ]:
from pyspark.sql.functions import count

In [ ]:
#circuito quantidade de vitoria por piloto
feature_store = df_F1.groupBy("circuit")\
.pivot("winner_name")\
.agg(count("*"))\
.fillna(0) #troca o null por 0



In [ ]:
feature_store.show(truncate=False)

+-----------------------------+-----------+----------+--------------+------------------+------------+-------------+------------+-------------+-----------+----------------+------------+---------------+--------------+----------+----------------+----------+---------------+-----------+-------------+------------+---------------+------------------+------------+------------+---------------+---------------+--------------+--------------+------------------+--------------------+-----------------+-----------+--------------+-----------------+---------------------+-------------+--------------+----------+------------+---------------+------------------+----------+------------+--------------------+---------------------+----------+-------------+-----------+---------+------------+-----------+------------+--------------+---------------+--------------+------------+-----------+---------------------+----------+----------+------------------+------------------+------------+--------------+------------+---------

In [ ]:
#ano x quantidade de vezes que o piloto ganhou
feature_store2 = df_F1.groupBy("year")\
.pivot("winner_name")\
.agg(count("*"))\
.orderBy("year", ascending=False)\
.fillna(0) #troca o null por 0


In [ ]:
feature_store2.show(truncate=False)

+----+-----------+----------+--------------+------------------+------------+-------------+------------+-------------+-----------+----------------+------------+---------------+--------------+----------+----------------+----------+---------------+-----------+-------------+------------+---------------+------------------+------------+------------+---------------+---------------+--------------+--------------+------------------+--------------------+-----------------+-----------+--------------+-----------------+---------------------+-------------+--------------+----------+------------+---------------+------------------+----------+------------+--------------------+---------------------+----------+-------------+-----------+---------+------------+-----------+------------+--------------+---------------+--------------+------------+-----------+---------------------+----------+----------+------------------+------------------+------------+--------------+------------+-----------+--------------+-------

In [ ]:
#ano x quantidade de vezes que cada equipe ganhou
feature_store3 = df_F1.groupBy("year")\
.pivot("team")\
.agg(count("*"))\
.orderBy("year", ascending=False)\
.fillna(0) #troca o null por 0

In [ ]:
feature_store3.show(truncate=False)

+----+----------+----------------+--------------+---+------------+-------------+----------------+------------------+-----------+--------------+------------+-------------+--------------+-------------+---------------+-------------+-------------------+-------+------------+-----+-----------+------------------+------------------------+-----------------+-----------+------------+------------------+---------+------------+----------+-----------+-------------+----------+--------+----------+-------+------------+-------------+----------------+-----------+--------+-------------+-----------+-------+-----------+-------------------------+---------------+---------------------+--------------------------+--------------------+-----------------------+-------------------------+-------+-----------+----------+-----------+------------+------------+-------+------------------+------------+-------------+--------------+----------------+---------+
|year|Alfa Romeo|AlphaTauri Honda|Alpine Renault|BRM|Benetton BMW|Be

In [ ]:
# equipe x quantdade de vezes que cada piloto ganhou
feature_store4 = df_F1.groupBy("team")\
.pivot("winner_name")\
.agg(count("*"))\
.orderBy("team", ascending=False)\
.fillna(0) #troca o null por 0

In [ ]:
feature_store4.show(truncate=False)

+--------------------------+-----------+----------+--------------+------------------+------------+-------------+------------+-------------+-----------+----------------+------------+---------------+--------------+----------+----------------+----------+---------------+-----------+-------------+------------+---------------+------------------+------------+------------+---------------+---------------+--------------+--------------+------------------+--------------------+-----------------+-----------+--------------+-----------------+---------------------+-------------+--------------+----------+------------+---------------+------------------+----------+------------+--------------------+---------------------+----------+-------------+-----------+---------+------------+-----------+------------+--------------+---------------+--------------+------------+-----------+---------------------+----------+----------+------------------+------------------+------------+--------------+------------+-----------+